In [2]:
import geopandas
import leafmap

C:\Users\khale\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
con = leafmap.connect_postgis(database='GIS', host='localhost', user='postgres', password='postgres')
sqlQ = "SELECT * FROM public.utility"
gdf = leafmap.read_postgis(sqlQ, con, geom_col='geometry')

In [4]:
prob_list = [[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry]
print(prob_list)
gdf.head(50)

[[30.004285, 31.295168], [31.49253, 30.061771], [31.49253, 30.061771], [31.036913, 31.361936], [31.036858, 31.362058], [31.49253, 30.061771], [31.627722, 30.132701], [31.627722, 30.132701]]


,id,sub_type,lat,lon,timestamp,geometry,effect,description,img,solved,solved_time,userid,type
0,1,Leakage,30.004285,31.295168,2022-03-20 19:00:46.486052,POINT (31.29517 30.00428),1,water,3bdeddc2b730a876.jpg,False,None,1,1
1,2,Water,31.492530,30.061771,2022-03-20 19:27:59.137023,POINT (30.06177 31.49253),1,Description,1234,False,None,1,1
2,3,Water,31.492530,30.061771,2022-03-20 19:28:06.836584,POINT (30.06177 31.49253),1,Description,1234,False,None,1,1
3,4,Leakage,31.036913,31.361936,2022-03-20 19:53:16.938415,POINT (31.36194 31.03691),1,nnn,8ed991ec6ebe6fdb.jpg,False,None,1,1
4,5,Leakage,31.036858,31.362058,2022-03-20 20:18:09.470694,POINT (31.36206 31.03686),1,x,7cc193f5304fd896.jpg,False,None,1,1
5,6,Water,31.492530,30.061771,2022-03-20 20:19:53.046769,POINT (30.06177 31.49253),1,Description,1234,False,None,1,1
6,7,Water,31.627722,30.132701,2022-03-20 20:20:32.314965,POINT (30.13270 31.62772),1,Description,1234,False,None,1,1
7,8,Water,31.627722,30.132701,2022-03-20 20:20:36.734742,POINT (30.13270 31.62772),1,Description,1234,False,None,1,1


In [ ]:
facilities = geopandas.read_file("../Data/Facilities/DemoCairo.gpkg", layer='facilities').to_crs("EPSG:3857") #Point
admin_poly = geopandas.read_file("../Data/Facilities/DemoCairo.gpkg", layer='NewCairoPolyDemo').to_crs("EPSG:3857") #Polygon
facilities.head(50)
Egy_fac= geopandas.read_file("../Data/Facilities/EgyptFacilities.gpkg", layer='facilities').to_crs("EPSG:3857") #Point
Egy_fac.head(50)

In [6]:
fac_And_Admin = geopandas.sjoin(Egy_fac,admin_poly[['adm3_ar', 'geometry']], how='left')
fac_And_Admin.rename_geometry('fac_geo', inplace=True)
fac_And_Admin_for_utility = fac_And_Admin.loc[fac_And_Admin['fclass']=='fire_station']
toto = fac_And_Admin_for_utility.to_crs("EPSG:4326")
toto['fac_lat'] = toto['fac_geo'].y
toto['fac_lon'] = toto['fac_geo'].x
geometry = geopandas.points_from_xy(toto.fac_lon, toto.fac_lat)

#facilities list
geo_df_list = [[point.xy[0][0], point.xy[1][0]] for point in toto.fac_geo]


#Insert prob to locations list
lastprob=prob_list[-1]
geo_df_list.insert(0,lastprob)

print(geo_df_list)


[[31.627722, 30.132701], [31.368957899999995, 30.067249200000003], [31.7441051, 30.3028631], [31.281973299999997, 30.0563657], [31.330913799999998, 30.094396000000003], [31.2237961, 30.183360299999997], [34.5172483, 28.495024899999997], [32.7207632, 29.591531700000004], [31.459280400000004, 30.883377999999997], [33.830379799999996, 27.239859], [30.760266600000005, 28.09504220000001], [30.941887199999996, 29.978313999999997], [34.778616799999995, 25.261348099999996], [31.185490000000005, 27.196521399999998], [31.395656400000004, 31.047463799999992], [31.388908199999992, 31.048263999999996], [31.3267534, 30.105157900000002], [29.9930528, 31.24089799999999], [31.31218459999999, 30.100378900000003], [31.2462273, 30.117107200000003], [34.109483399999995, 31.232817099999995], [31.832260899999998, 31.5087551], [27.109660600000005, 30.781374199999995]]


In [7]:
from xml.etree.ElementTree import tostring
import requests
import json

In [8]:
z=[]
for i in range(len(geo_df_list)-1):
    z.append(i+1)
print(z)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]


In [9]:
#od matrix

import requests

body = {"locations":geo_df_list,"destinations":[0],"metrics":["distance","duration"],"sources":z,"units":"m"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf62487ad6039a4daa436d88a8a4ff481a40f9',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)
print(call.text)

200 OK
{"durations":[[1499.92],[1893.47],[1840.05],[1728.94],[2529.89],[23775.14],[6800.65],[5324.43],[16319.2],[11627.43],[3722.55],[26307.25],[15929.49],[6326.15],[6333.22],[1869.84],[9971.27],[1850.64],[2347.24],[13699.33],[9305.5],[20773.29]],"distances":[[34173.84],[36542.29],[42674.41],[38600.59],[48416.37],[518449.53],[163652.78],[106865.02],[453067.91],[305475.81],[83221.78],[704192.69],[425988.19],[126817.52],[127130.91],[37635.98],[273643.38],[40696.52],[48049.91],[322528.97],[222992.25],[535493.13]],"destinations":[{"location":[31.627968,30.131661],"snapped_distance":117.98}],"sources":[{"location":[31.369141,30.067437],"snapped_distance":27.34},{"location":[31.744056,30.302858],"snapped_distance":4.72},{"location":[31.282053,30.055799],"snapped_distance":63.5},{"location":[31.330827,30.09431],"snapped_distance":12.76},{"location":[31.224005,30.183281],"snapped_distance":21.88},{"location":[34.515599,28.495021],"snapped_distance":161.2},{"location":[32.720924,29.591414],"sna

In [10]:
res = json.loads(call.text)
x=res['durations']

durationList=[]
for i in x:
    m=i[0]
    durationList.append(m)
durationList.sort()
minDuration=durationList[0]
print(minDuration)

index = x.index([minDuration])

sourceList=res['sources']
source=sourceList[index]
closestFac=source['location']

print(closestFac)


1499.92
[31.369141, 30.067437]


In [11]:
q=closestFac[0]
w=closestFac[1]

r=lastprob[0]
t=lastprob[1]


In [23]:
import pprint
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
}
call = requests.get('https://api.openrouteservice.org/v2/directions/driving-car?api_key=5b3ce3597851110001cf62487ad6039a4daa436d88a8a4ff481a40f9&start={},{}&end={},{}'.format(q,w,r,t), headers=headers)

print(call.status_code, call.reason)
print(call.text)


200 OK
{"type":"FeatureCollection","features":[{"bbox":[31.368233,30.066073,31.643503,30.139081],"type":"Feature","properties":{"segments":[{"distance":34173.8,"duration":1499.9,"steps":[{"distance":39.2,"duration":9.4,"type":11,"instruction":"Head east","name":"-","way_points":[0,1]},{"distance":163.0,"duration":13.0,"type":1,"instruction":"Turn right","name":"-","way_points":[1,5]},{"distance":1036.5,"duration":74.6,"type":7,"instruction":"Enter the roundabout and take the 1st exit onto شارع ابو بكر الصديق","name":"شارع ابو بكر الصديق","exit_number":1,"way_points":[5,46]},{"distance":145.0,"duration":11.6,"type":1,"instruction":"Turn right onto شارع الثانوية الجويه","name":"شارع الثانوية الجويه","way_points":[46,58]},{"distance":74.5,"duration":8.6,"type":6,"instruction":"Continue straight onto شارع الصاعقه","name":"شارع الصاعقه","way_points":[58,62]},{"distance":678.5,"duration":62.5,"type":13,"instruction":"Keep right onto شارع الورشه","name":"شارع الورشه","way_points":[62,74]},{"d

In [24]:
#Next Is Testing


In [19]:
#od matrix

import requests

body = {"locations":[[31.45909309387207,30.051265218888904],[31.458749771118164,30.022212346219188],[31.45909309387207,30.03172419884694]],"destinations":[0],"metrics":["distance","duration"],"sources":[1,2],"units":"m"}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf62487ad6039a4daa436d88a8a4ff481a40f9',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)
print(call.text)

200 OK
{"durations":[[675.66],[326.16]],"distances":[[7829.26],[4137.25]],"destinations":[{"location":[31.459046,30.051266],"snapped_distance":4.52}],"sources":[{"location":[31.458766,30.022212],"snapped_distance":1.6},{"location":[31.459182,30.031698],"snapped_distance":9.06}],"metadata":{"attribution":"openrouteservice.org | OpenStreetMap contributors","service":"matrix","timestamp":1647949234559,"query":{"locations":[[31.45909309387207,30.051265218888904],[31.458749771118164,30.022212346219188],[31.45909309387207,30.03172419884694]],"profile":"driving-car","responseType":"json","metricsStrings":["DISTANCE","DURATION"],"sources":["1","2"],"destinations":["0"],"metrics":["distance","duration"],"units":"m"},"engine":{"version":"6.7.0","build_date":"2022-02-18T19:37:41Z","graph_date":"2022-03-13T11:00:24Z"}}}


In [20]:
res = json.loads(call.text)
x=res['durations']


durationList=[]
for i in x:
    m=i[0]
    durationList.append(m)
durationList.sort()
minDuration=durationList[0]
print(minDuration)

index = x.index([minDuration])

sourceList=res['sources']
source=sourceList[index]
closestFac=source['location']

print(closestFac)


326.16
[31.459182, 30.031698]


In [23]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
}
call = requests.get('https://api.openrouteservice.org/v2/directions/driving-car?api_key=5b3ce3597851110001cf62487ad6039a4daa436d88a8a4ff481a40f9&start=31.459182, 30.031698&end=31.45909309387207,30.051265218888904', headers=headers)

print(call.status_code, call.reason)
print(call.text)

200 OK
{"type":"FeatureCollection","features":[{"bbox":[31.450058,30.030831,31.459182,30.052756],"type":"Feature","properties":{"segments":[{"distance":4137.2,"duration":326.2,"steps":[{"distance":140.4,"duration":9.8,"type":11,"instruction":"Head south on محور احمد شوقى الجانبى","name":"محور احمد شوقى الجانبى","way_points":[0,7]},{"distance":10.2,"duration":3.7,"type":0,"instruction":"Turn left","name":"-","way_points":[7,8]},{"distance":690.5,"duration":45.2,"type":1,"instruction":"Turn right onto شارع التسعين الشمالى","name":"شارع التسعين الشمالى","way_points":[8,14]},{"distance":39.5,"duration":2.8,"type":13,"instruction":"Keep right onto شارع السيدة زينب","name":"شارع السيدة زينب","way_points":[14,15]},{"distance":3076.9,"duration":221.5,"type":6,"instruction":"Continue straight onto شارع السيدة زينب","name":"شارع السيدة زينب","way_points":[15,72]},{"distance":38.6,"duration":9.3,"type":13,"instruction":"Keep right","name":"-","way_points":[72,75]},{"distance":141.2,"duration":33.